In [8]:
import pandas as pd
from skimage.measure import regionprops_table
from imageio.v3 import imread, imwrite

def df_from_cellpose_mask(mask):
    props_cellpose = regionprops_table(
        mask,
        properties=[
            "label",
            "area",
            "centroid",
            "eccentricity",
            "bbox",
            "image",
            "perimeter",
            "feret_diameter_max",
        ],
    )
    df_cellpose = pd.DataFrame(props_cellpose)
    return df_cellpose

file_name = "WT_4_GAS_1.ome.tif"
wsi = imread("TG_Cut/WT_4_GAS_1.ome.tif")
cellpose_mask = imread("TG_Cut/WT_4_GAS_1.ome_cp_masks.tif")
df = df_from_cellpose_mask(cellpose_mask)
# Drop row from df if area column value is inferior to 1000
df = df[df["area"] > 1000]
# import glob
# from imageio.v3 import imread, imwrite
# all_wsi = glob.glob("TG_Cut/*.ome.tif")
# all_masks = [wsi.replace(".ome.tif", ".ome_cp_masks.tif") for wsi in all_wsi]
# import math
# import matplotlib.pyplot as plt
# def extract_single_image(raw_image, df_props, index, erosion=None):
#     single_entity_img = raw_image[
#         df_props.iloc[index, 5] : df_props.iloc[index, 7],
#         df_props.iloc[index, 6] : df_props.iloc[index, 8],
#     ].copy()
#     surface_area = df_props.iloc[index, 1]
#     cell_radius = math.sqrt(surface_area / math.pi)
#     single_entity_mask = df_props.iloc[index, 9].copy()

#     if erosion is not None:
#         erosion_size = cell_radius * (
#             erosion / 100
#         )  # Erosion in percentage of the cell radius
#         for i in range(int(erosion_size)):
#             single_entity_mask = binary_erosion(
#                 single_entity_mask, out=single_entity_mask
#             )

#     single_entity_img[~single_entity_mask] = 0
#     return single_entity_img

# for img_idx in range(len(all_wsi)):
#     wsi = imread(all_wsi[img_idx])
#     mask = imread(all_masks[img_idx])
#     file_name = all_wsi[img_idx].split("/")[-1]
#     root_file_name = file_name[: -len(".ome.tif")]
#     df = df_from_cellpose_mask(mask)
#     df = df[df["area"] > 1000]
#     for i in range(len(df)):
#         single_img = extract_single_image(wsi, df, i, erosion=None)
#         file_path = f"TG_Cut/TG_all_cells/unclassified/{i}_{root_file_name}.tif"
#         imwrite(file_path, single_img)

,label,area,centroid-0,centroid-1,eccentricity,bbox-0,bbox-1,bbox-2,bbox-3,image,perimeter,feret_diameter_max
0,1,9918,124.093466,7090.208106,0.850952,59,7033,214,7136,"[[False, False, False, False, False, False, Fa...",441.717821,157.305435
1,2,8858,145.550914,7029.629939,0.927333,71,6975,230,7083,"[[False, False, False, False, False, False, Fa...",434.759451,164.453641
2,3,5519,121.798333,6834.645044,0.632403,86,6783,163,6880,"[[False, False, False, False, False, False, Fa...",301.030483,110.535062
3,4,18763,176.103448,7192.154400,0.747389,87,7127,281,7261,"[[False, False, False, False, False, False, Fa...",568.617316,204.012255
4,5,9772,159.104687,6901.808432,0.850312,88,6861,238,6951,"[[False, False, False, False, False, False, Fa...",415.320851,151.076140
...,...,...,...,...,...,...,...,...,...,...,...,...
5393,5394,1008,8174.064484,515.109127,0.868894,8160,490,8187,545,"[[False, False, False, False, False, False, Fa...",139.497475,55.326305
5394,5395,1681,8198.771565,511.192148,0.881728,8182,481,8215,545,"[[False, False, False, False, False, False, Fa...",178.669048,71.554175
5395,5396,128,8187.921875,460.164062,0.921636,8183,447,8196,472,"[[False, False, False, False, False, False, Fa...",54.834524,26.248809
5396,5397,163,8198.061350,449.552147,0.989830,8188,435,8211,473,"[[False, False, False, False, False, False, Fa...",89.669048,43.908997


In [35]:
from pigeon import annotate
from PIL import Image
from IPython.display import display
import glob
all_files = glob.glob("TG_Cut/TG_all_cells/unclassified/*.tif")
annotations = annotate(
  all_files,
  options=["control", "central aggregates", "periphery aggregates", "reject/remove"],
  display_fn=lambda filename: display(Image.open(filename, 'r'))
)

HTML(value='0 examples annotated, 103378 examples left')

Output()

In [34]:
# move each file to the corresponding folder
import shutil
import os
for file, annotation in zip(all_files, annotations):
    if annotation == "reject/remove":
        shutil.move(file, f"TG_Cut/TG_all_cells/reject/")
    if annotation == "periphery aggregates":
        shutil.move(file, f"TG_Cut/TG_all_cells/aggpreiph/")
    if annotation == "central aggregates":
        shutil.move(file, f"TG_Cut/TG_all_cells/aggcenter/")
    if annotation == "control":
        shutil.move(file, f"TG_Cut/TG_all_cells/control/")

[('TG_Cut/TG_all_cells/unclassified/684_WT_4_TA_4.tif', 'control'),
 ('TG_Cut/TG_all_cells/unclassified/2770_WT_23_GAS_2.tif',
  'periphery aggregates'),
 ('TG_Cut/TG_all_cells/unclassified/340_WT_8_GAS_3.tif',
  'periphery aggregates'),
 ('TG_Cut/TG_all_cells/unclassified/3345_WT_23_GAS_1.tif',
  'central aggregates'),
 ('TG_Cut/TG_all_cells/unclassified/1889_WT_23_TA_4.tif', 'control'),
 ('TG_Cut/TG_all_cells/unclassified/600_WT_22_GAS_2.tif',
  'central aggregates'),
 ('TG_Cut/TG_all_cells/unclassified/1580_WT_4_GAS_3.tif', 'control'),
 ('TG_Cut/TG_all_cells/unclassified/3707_WT_23_GAS_1.tif',
  'central aggregates')]